# Appendix 1 Customizing LLM based on LangChain

LangChain provides an efficient development framework for developing custom applications based on LLM, which makes it easy for developers to quickly stimulate the powerful capabilities of LLM and build LLM applications. LangChain also supports a variety of large models, and has built-in call interfaces for large models such as OpenAI and LLAMA. However, LangChain does not have all large models built in. It provides powerful scalability by allowing users to customize LLM types.

In this section, we take the Baidu Wenxin large model as an example to describe how to customize LLM based on LangChain, so that the applications built on LangChain can support domestic large models such as Baidu Wenxin and iFlytek Spark.

This section involves relatively more technical details of LangChain and large model calls. Students with energy can learn to deploy, and if you don’t have energy, you can directly use the subsequent code to support calls.

To implement custom LLM, you need to define a custom class that inherits from LangChain's LLM base class, and then define two functions: 
① _call method, which accepts a string and returns a string, which is the core call of the model; 
② _identifying_params method, used to print LLM information. 

First, we import the required third-party libraries:

In [12]:
import json
import time
from typing import Any, List, Mapping, Optional, Dict, Union, Tuple
import requests
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.utils import get_from_dict_or_env
from pydantic import Field, model_validator

Since Baidu Wenxin uses dual secret keys for authentication, users need to first obtain access_token based on API_Key and Secret_Key, and then use access_token to call the model (see "3. Calling Baidu Wenxin" for details). Therefore, we need to define a get_access_token method to obtain access_token:

In [4]:
def get_access_token(api_key : str, secret_key : str):
    """
    使用 API Key，Secret Key 获取access_token，替换下列示例中的应用API Key、应用Secret Key
    """
# Specify the URL
    url = f"https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id={api_key}&client_secret={secret_key}"
# Set up POST access
    payload = json.dumps("")
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
# Access the corresponding access_token of the account through POST
    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json().get("access_token")

Next, we define a custom LLM class that inherits from the LLM class:

In [5]:
# Inherited from langchain.llms.base.LLM
class Wenxin_LLM(LLM):
# Native interface address
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant"
# The default model is ERNIE-Bot-turbo, which is currently known as the Baidu Wenxin model.
    model_name: str = Field(default="ERNIE-Bot-turbo", alias="model")
# Access delay limit
    request_timeout: Optional[Union[float, Tuple[float, float]]] = None
# Temperature Coefficient
    temperature: float = 0.1
# API_Key
    api_key: str = None
# Secret_Key
    secret_key : str = None
# access_token
    access_token: str = None
# Required optional parameters
    model_kwargs: Dict[str, Any] = Field(default_factory=dict)


The above initialization covers the parameters we usually use. You can also add more parameters according to actual needs and Wenxin's API.

Next, we implement an initialization method init_access_token, which is called when the model's access_token is empty:

In [6]:
def init_access_token(self):
    if self.api_key != None and self.secret_key != None:
# Both keys must be non-empty to obtain access_token
        try:
            self.access_token = get_access_token(self.api_key, self.secret_key)
        except Exception as e:
            print(e)
            print("获取 access_token 失败，请检查 Key")
    else:
        print("API_Key 或 Secret_Key 为空，请检查 Key")

Next we implement the core method - calling the model API:

In [7]:
def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
# Except for the prompt parameter, other parameters are not used, but when we call it through LangChain, these parameters will be passed in, so they must be set
# If access_token is empty, initialize access_token
    if self.access_token == None:
        self.init_access_token()
# API call url
    url = "https://aip.baidubce.com/rpc/2.0/ai_custom/v1/wenxinworkshop/chat/eb-instant?access_token={}".format(self.access_token)
# Configure POST parameters
    payload = json.dumps({
        "messages": [
            {
                "role": "user",# user prompt
                "content": "{}".format(prompt)# 输入的 prompt
            }
        ],
        'temperature' : self.temperature
    })
    headers = {
        'Content-Type': 'application/json'
    }
# Make a request
    response = requests.request("POST", url, headers=headers, data=payload, timeout=self.request_timeout)
    if response.status_code == 200:
# The returned value is a Json string
        js = json.loads(response.text)
        return js["result"]
    else:
        return "请求失败"

Then we also need to define the description method of the model:

In [9]:
# First define a method that returns default parameters
@property
def _default_params(self) -> Dict[str, Any]:
"""Get the default parameters for calling the Ennie API."""
    normal_params = {
        "temperature": self.temperature,
        "request_timeout": self.request_timeout,
        }
    return {**normal_params}


@property
def _identifying_params(self) -> Mapping[str, Any]:
"""Get the identifying parameters."""
    return {**{"model_name": self.model_name}, **self._default_params}

Through the above steps, we can define the calling method of Baidu Wenxin based on LangChain. We encapsulate this code in the wenxin_llm.py file and use this LLM directly in the notebook that describes how to call Baidu Wenxin.